Before running the script make sure you have added a shortcut to the [NOAH Datasets](https://drive.google.com/drive/folders/1ALE4-E9c-4AGjm1fqiPprWHrLUskeY9o?usp=share_link) into your Google Drive...

In [3]:
# import tensorflow as tf

# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [4]:
# Mount data from drive
# from google.colab import drive
# drive.mount('/content/drive')

*Making a copy of NOAH dataset on Google Drive.* First, `cd` into the NOAH dataset shortcut by running the cell below. The path to it might differ for you but in my case I added it into my GDrive's root folder. 

The format for the command is: 


```
%cd /path/to/NOAH/shortcut
```

Upon running, it should show the exact path to the symbolic link. Copy it! Mine looks like this: 

`/content/drive/.shortcut-targets-by-id/1ALE4-E9c-4AGjm1fqiPprWHrLUskeY9o/NOAH Downloads`

In [5]:
#%cd /content/drive/MyDrive/NOAH Downloads

Next, create a new folder in your Google Drive where we will copy the contents of the NOAH dataset to. Mine is in the path: 

`/content/drive/MyDrive/noah`

(I don't use Colab that much so I don't mind cluttering up my root folder lol)

Run the cell below to copy from the shortcut into your desired destination folder. This has to be done because files inside shortcuts are read-only. 

`cp -r '/sym/link/path/.' 'path/to/desired/directory'`

In [6]:
#%cp -r '/content/drive/.shortcut-targets-by-id/1ALE4-E9c-4AGjm1fqiPprWHrLUskeY9o/NOAH Downloads/.' '/content/drive/MyDrive/noah'

Lastly, under your desired category, copy the contents of the ph_cities_v2 folder from our repo. 

The path in the repo is this: 

`philippines-chapter-urban-vunerability/src/data/geolocation/ph_cities_joined_v2/`


In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import shutil
import geopandas as gpd
import os
import zipfile

ModuleNotFoundError: No module named 'geopandas'

## Helper functions

In [8]:
def unzip_path(path):
  zipfile.ZipFile(path).extractall(temp_folder)

In [9]:
def delete_zip_folder():
  for filename in os.listdir(temp_folder):
    file_path = os.path.join(temp_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [10]:
def map_to_city(path_to_dir, dirs):
  for dir in dirs:
    print(f"Merging {dir}...")
    path = Path(path_to_dir, dir)
    zip_files = os.listdir(path)

    for i in range(len(zip_files)):

      print(f"Progress: {i+1}/{len(zip_files)}")

      print(f"Unzipping {zip_files[i]}...")
      unzip_path(Path(path, zip_files[i]))

      print(f"Reading {zip_files[i]}...")
      if i == 0:
        gdf = gpd.read_file(Path(path,zip_files[i]))
        dissolve_col = [col for col in gdf.columns if col != "geometry"][0]
      else:
        temp_gdf = gpd.read_file(Path(path,zip_files[i]))
        gdf = pd.concat([gdf, temp_gdf])

        # Dissolving in HAZ

        gdf = gdf.dissolve(by=dissolve_col, aggfunc="sum")
        gdf = gdf.reset_index()

      print(f"Deleting {zip_files[i]} artifacts...")
      delete_zip_folder()
      print("========")

    destination_dir = Path(path, f"{dir}-merged")
    destination_dir.mkdir(parents=True, exist_ok=True)

    gdf.to_parquet(Path(destination_dir, f"{dir}.parquet"))

    merged = gpd.overlay(df, gdf, how='intersection').reset_index()
    merged.to_parquet(Path(destination_dir, f"{dir}-by-city-munic.parquet"))

In [11]:
# Create temp_folder to hold unarchived zip-files
temp_folder = Path('../../data/geolocation/temp_folder')
temp_folder.mkdir(parents=True, exist_ok=True)

In [12]:
# Import city/municipality df
# This is the path to the shp file in the
# ph_cities_joined_v2 we pasted earlier.
df = gpd.read_file('../../data/dti-competitive-index/2-merged/1-shape_files/complete_dataset_with_geo_data.shp')

In [24]:
df.head(1)

,name,geometry
0,Aborlan,"MULTIPOLYGON (((118.57998 9.37215, 118.57976 9..."


In [14]:
keep_cols = ["name", "geometry"]
df = df[keep_cols]

In [22]:
df.head(1)

,name,geometry
0,Aborlan,"MULTIPOLYGON (((118.57998 9.37215, 118.57976 9..."


## Storm Surge

In [16]:
PATH_TO_STORMSURGE_DIR = Path('G:/.shortcut-targets-by-id/1ALE4-E9c-4AGjm1fqiPprWHrLUskeY9o/NOAH Downloads/Storm Surge')
storm_surge_advisory_dirs = [f"StormSurgeAdvisory{i}" for i in np.arange(1,5)]

In [17]:
map_to_city(PATH_TO_STORMSURGE_DIR, storm_surge_advisory_dirs)

Merging StormSurgeAdvisory1...
Progress: 1/69
Unzipping Zambales.zip...
Reading Zambales.zip...
Deleting Zambales.zip artifacts...
Progress: 2/69
Unzipping CamarinesSur.zip...
Reading CamarinesSur.zip...
Deleting CamarinesSur.zip artifacts...
Progress: 3/69
Unzipping Quezon.zip...
Reading Quezon.zip...
Deleting Quezon.zip artifacts...
Progress: 4/69
Unzipping Aurora.zip...
Reading Aurora.zip...
Deleting Aurora.zip artifacts...
Progress: 5/69
Unzipping Batanes.zip...
Reading Batanes.zip...
Deleting Batanes.zip artifacts...
Progress: 6/69
Unzipping LaUnion.zip...
Reading LaUnion.zip...
Deleting LaUnion.zip artifacts...
Progress: 7/69
Unzipping Batangas.zip...
Reading Batangas.zip...
Deleting Batangas.zip artifacts...
Progress: 8/69
Unzipping CamarinesNorte.zip...
Reading CamarinesNorte.zip...


KeyboardInterrupt: 

## Flood

In [35]:
PATH_TO_FLOOD_DIR = Path('G:/.shortcut-targets-by-id/1ALE4-E9c-4AGjm1fqiPprWHrLUskeY9o/NOAH Downloads/Flood/')
flood_dirs = ["5yr", "25yr", "100yr"]

In [36]:
map_to_city(PATH_TO_FLOOD_DIR, flood_dirs)

Merging 5yr...
Progress: 1/66
Unzipping IlocosNorte.zip...
Reading IlocosNorte.zip...
Deleting IlocosNorte.zip artifacts...
Progress: 2/66
Unzipping LaUnion.zip...
Reading LaUnion.zip...


## Landslide Hazards

In [11]:
PATH_TO_LANDSLIDE_HAZ_DIR = Path('G:/.shortcut-targets-by-id/1ALE4-E9c-4AGjm1fqiPprWHrLUskeY9o/NOAH Downloads/Landslide')
landslide_haz_dirs = ["LandslideHazards"]

In [12]:
map_to_city(PATH_TO_LANDSLIDE_HAZ_DIR, landslide_haz_dirs)

Merging LandslideHazards...
Progress: 1/82
Unzipping IlocosNorte.zip...
Reading IlocosNorte.zip...
Deleting IlocosNorte.zip artifacts...
Progress: 2/82
Unzipping Pangasinan.zip...
Reading Pangasinan.zip...
